In [1]:
import pymeshlab
import tqdm
import math
import os

from utils import *

In [ ]:
MododelNet_dir = "/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled"

paths_mesh = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_files_ModelNet40_centered_scaled.txt')

In [ ]:
def compute_face_area_stats(mesh):
    """Compute mean, standard deviation, and CV of face areas."""
    face_areas = mesh.face_matrix() # Get the areas of all faces
    mean_area = np.mean(face_areas)
    std_dev = np.std(face_areas)
    cv = std_dev / mean_area if mean_area != 0 else 0
    return mean_area, std_dev, cv

In [ ]:
max_iterations = 20  # Safety limit for iterations
cv_threshold = 0.05  # Coefficient of variation threshold for uniformity

In [ ]:
paths_mesh[:5]

In [ ]:
results = []  # List to store results
for path_mesh in tqdm.tqdm(paths_mesh):
    try :
        # categorie = "radio"
        # type = 'train'
        # num = '0001'
        # path_mesh = "/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled/"+categorie+"/"+type+"/"+categorie+"_"+num+"_SMPLER_centered_scaled.obj"
        # Create the directory, skip if it already exists
        # os.makedirs('tmp/'+categorie, exist_ok=True)
        
        # Load the mesh
        ms = pymeshlab.MeshSet()
        ms.load_new_mesh(path_mesh)
        name_mesh = os.path.basename(path_mesh)
        
        # # Diagonal
        # # Compute the bounding box dimensions
        # bbox = ms.current_mesh().bounding_box()
        # min_point = bbox.min()
        # max_point = bbox.max()

        # # Calculate the diagonal length
        # diagonal = math.sqrt(
        #     (max_point[0] - min_point[0])**2 +  # X-axis difference
        #     (max_point[1] - min_point[1])**2 +  # Y-axis difference
        #     (max_point[2] - min_point[2])**2)    # Z-axis difference

        cv_values = []  # List to store CV values
        stop_iteration = None  # To store the iteration where CV repeats
        
        ## Automatic determine iteration param
        for iter in range(3, max_iterations+1):
            # load the mesh
            ms.load_new_mesh(path_mesh)
            # Apply isotropic remeshing : default param are the same in meshlab
            ms.meshing_isotropic_explicit_remeshing(iterations=iter)
            # Compute face area statistics
            mean_area, std_dev, cv = compute_face_area_stats(ms.current_mesh()); #print(f"Iteration {iter}: Mean Area = {mean_area}, Std Dev = {std_dev}, CV = {cv}, {np.round(cv,3)}")
            # Store CV value in the list
            cv_values.append(np.round(cv,3))
            # Check if the CV is below the threshold
            if len(cv_values) >= 3 and (cv_values[-2] == cv_values[-3] == cv_values[-4]):
                #print(f"CV value {cv} has repeated for the first time after iteration {iter}.")
                stop_iteration = iter-3; #print(f"Stop iteration: {stop_iteration}")
                break

        # Save the remeshed result
        #ms.save_current_mesh(os.path.join(MododelNet_dir, name_mesh+"_remeshing_iso_iter"+str(stop_iteration)+".obj"))
        results.append(("ok", path_mesh))
        
    except Exception as e: results.append(("pbl", path_mesh))
          
# Write results to file
with open("/home/pelissier/These-ATER/Papier_international3/Dataset/error_run1_remeshing_iso.txt", "w") as file:
    for name, path in results:
        file.write(f"{name}: {path}\n")
        